In [1]:
!git clone https://github.com/asellam/sitrex.git
%cd sitrex

fatal: destination path 'sitrex' already exists and is not an empty directory.
/content/sitrex


In [2]:
!pip install -r requirements.txt

In [3]:
import kagglehub

path = kagglehub.dataset_download('hasyimabdillah/workoutfitness-video')

100%|██████████| 4.32G/4.32G [00:39<00:00, 116MB/s] 

Extracting files...


In [4]:
import sys
sys.path.append('.')
# Import necessary libraries
import os
from sklearn.model_selection import train_test_split
from sitrex.preprocessing import calculate_angles, get_angle_points
from sitrex.model import angle_usefulness_model, angle_similarity_model, TQDMProgressBar
import tensorflow as tf
import numpy as np

# EarlyStop save path for the Angle Similarity Transformer
MODEL_SAVE_PATH_USEFULNESS = './weights/angle_similarity_model_L300.h5'
# EarlyStop save path for the Angle Usefulness Transformer
MODEL_SAVE_PATH_SIMILARITY = './weights/angle_usefulness_model_L800.h5'
# Batch Size
batch_size = 32

# Load the trained Siamese model
similarity_model = angle_similarity_model(maxlen=300, module='transformer', lr=1e-4)
similarity_model.load_weights(MODEL_SAVE_PATH_SIMILARITY)
usefulness_model = angle_usefulness_model(maxlen=800, module='transformer', lr=1e-4)
usefulness_model.load_weights(MODEL_SAVE_PATH_USEFULNESS)
print("Models loaded successfully.")

Models loaded successfully.


In [6]:
import numpy as np

def normalize_landmarks(landmarks):
    if not landmarks or len(landmarks) < 33:
        return []

    # Body center calculation (using hips)
    left_hip = landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value]
    center = {
        'x': (left_hip.x + right_hip.x) / 2,
        'y': (left_hip.y + right_hip.y) / 2,
        'z': (left_hip.z + right_hip.z) / 2
    }

    # Scale calculation (shoulder width)
    left_shoulder = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value]
    scale = abs(left_shoulder.x - right_shoulder.x) or 1e-5

    normalized = []
    for lm in landmarks:
        normalized.append({
            'x': (lm.x - center['x']) / scale,
            'y': (lm.y - center['y']) / scale,
            'z': lm.z - center['z'],
            'visibility': lm.visibility
        })
    return normalized

In [7]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
from tqdm import tqdm

coach_video = os.path.join(path, 'romanian deadlift/romanian deadlift_10.mp4')
athlete_video = os.path.join(path, 'deadlift/deadlift_25.mp4')
poser = mp.solutions.pose.Pose(
            static_image_mode=False,
            model_complexity=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.7
        )

cap = cv2.VideoCapture(coach_video)
T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
coach_frames = np.empty((T, H, W, 3), np.uint8)
coach_angles = np.empty((T, 23), np.uint8)
for t in tqdm(range(T)):
    success, frame = cap.read()
    coach_frames[t, :, :, :] = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = poser.process(frame)
    landmarks = result.pose_landmarks.landmark
    angles = calculate_angles(normalize_landmarks(landmarks))
    names = sorted(list(angles.keys()))
    angles = [angles[name] for name in names]
    coach_angles[t, :] = angles

100%|██████████| 140/140 [00:16<00:00,  8.39it/s]


In [8]:
coach = np.empty((1, usefulness_model.input_shape[1], usefulness_model.input_shape[2]), np.float32)
if coach_angles.shape[0] < usefulness_model.input_shape[1]:
    coach[0, :coach_angles.shape[0], :] = coach_angles[:, :]
    coach[0, coach_angles.shape[0]:, :] = 0
else:
    coach[0, :, :] = coach_angles[:usefulness_model.input_shape[1], :]
usefulness = np.where(usefulness_model.predict(coach)[0] < 0.5, 0, 1)
print(usefulness)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step
[1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1]


In [9]:
cap = cv2.VideoCapture(athlete_video)
T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FPS = int(cap.get(cv2.CAP_PROP_FPS))
athlete_frames = np.empty((T, H, W, 3), np.uint8)
athlete_angles = np.empty((T, 23), np.uint8)
athlete_landmarks = []
for t in tqdm(range(T)):
    success, frame = cap.read()
    if success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = poser.process(frame)
        landmarks = result.pose_landmarks.landmark
        angles = calculate_angles(normalize_landmarks(landmarks))
        names = sorted(list(angles.keys()))
        angles = [angles[name] for name in names]
        athlete_frames[t, :, :, :] = frame
        athlete_angles[t, :] = angles
        athlete_landmarks.append(landmarks)

100%|██████████| 938/938 [01:54<00:00,  8.22it/s]


In [12]:
coach = np.empty((athlete_angles.shape[0], similarity_model.input_shape[0][1], similarity_model.input_shape[0][2]), np.float32)
if coach_angles.shape[0] < similarity_model.input_shape[0][1]:
    for t in range(athlete_angles.shape[0]):
        coach[t, :coach_angles.shape[0], :] = coach_angles[:, :]
        coach[t, coach_angles.shape[0]:, :] = 0
else:
    for t in range(athlete_angles.shape[0]):
        coach[t, :, :] = coach_angles[:similarity_model.input_shape[0][1], :]

athlete = np.empty((athlete_angles.shape[0], similarity_model.input_shape[1][1], similarity_model.input_shape[1][2]), np.float32)
for t in range(athlete_angles.shape[0]):
    if t < similarity_model.input_shape[1][1]:
        athlete[t, :t, :] = athlete_angles[:t, :]
        athlete[t, t:, :] = 0
    else:
        athlete[t, :, :] = athlete_angles[t-similarity_model.input_shape[1][1]:t, :]

similarity = similarity_model.predict([coach, athlete])

30/30 ━━━━━━━━━━━━━━━━━━━━ 12s 396ms/step


In [13]:
from PIL import Image, ImageDraw

def standardize_landmarks(landmarks):
    if not landmarks or len(landmarks) < 33:
        return []
    normalized = []
    for lm in landmarks:
        normalized.append({
            'x': lm.x,
            'y': lm.y,
            'z': lm.z,
            'visibility': lm.visibility
        })
    return normalized

def draw_pi_slice(draw, center, point1, point2, fill):
    radius = np.linalg.norm(point1 - center)
    bbox = [
        center[0] - radius, center[1] - radius,
        center[0] + radius, center[1] + radius
    ]

    def angle_between(p, center):
        dx = p[0] - center[0]
        dy = p[1] - center[1]
        angle = np.degrees(np.arctan2(dy, dx)) % 360
        return angle

    start_angle = angle_between(point1, center)
    end_angle = angle_between(point2, center)
    if (end_angle - start_angle) % 360 > 180:
        end_angle, start_angle = start_angle, end_angle
    draw.pieslice(bbox, start=start_angle, end=end_angle, fill=fill)

# Define the codec and create VideoWriter object
w, h = athlete_frames.shape[2], athlete_frames.shape[1]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
out = cv2.VideoWriter('output_video.mp4', fourcc, FPS, (w, h))

# Write images to video
for t in tqdm(range(athlete_frames.shape[0])):
    landmarks = athlete_landmarks[t]
    frame = Image.fromarray(athlete_frames[t])
    draw = ImageDraw.Draw(frame)
    angle_points = get_angle_points(standardize_landmarks(landmarks))
    for i in range(len(angle_points)):
        if usefulness[i] >= 0.5:
            a, b, c, d, e = angle_points[names[i]]
            r, g = int(255 * (1 - similarity[t, i])), int(255 * similarity[t, i])
            draw.line([a[0]*w, a[1]*h, b[0]*w, b[1]*h], fill=(r, g, 0), width=5)
            draw.line([a[0]*w, a[1]*h, c[0]*w, c[1]*h], fill=(r, g, 0), width=5)
            draw_pi_slice(draw,
                          np.array([a[0]*w, a[1]*h]),
                          np.array([d[0]*w, d[1]*h]),
                          np.array([e[0]*w, e[1]*h]),
                          (r, g, 0))
    out.write(cv2.cvtColor(np.array(frame), cv2.COLOR_BGR2RGB))  # Add frame

# Release everything
out.release()
print("Video saved as output_video.mp4")

  0%|          | 0/938 [00:00<?, ?it/s]


NameError: name 'normalize_angle_points' is not defined